In [ ]:
!ls ../../../tft_torch/

ADF  miniconda3  ml_phenology  notebooks  phenology-ml-clm  tft-torch


In [1]:
import torch
import os
from tft_torch.tft import TemporalFusionTransformer
import pickle
from omegaconf import OmegaConf, DictConfig
import torch
import tft_torch

print(tft_torch.__file__)

/glade/u/home/ayal/tft-torch/tft_torch/__init__.py


In [2]:
checkpoint= "/glade/u/home/ayal/phenology-ml-clm/models/US_MMS_1ep_nofut_061325.pth"
filename= "/glade/u/home/ayal/phenology-ml-clm/data/USMMS_to_tft_06132025.pkl"
configuration = {'optimization':
                {
                    'batch_size': {'training': 8, 'inference': 8},# both weere 64 before
                    'learning_rate': 1e-4,#was 0.001
                    'max_grad_norm': 1.0,
                }
                ,
                'model':
                {
                    'dropout': 0.2,#was 0.05 before
                    'state_size': 160,
                    'output_quantiles': [0.1, 0.5, 0.9],
                    'lstm_layers': 4,#was 2
                    'attention_heads': 4 #was 4 #then 6
                },
                # these arguments are related to possible extensions of the model class
                'task_type':'regression',
                'target_window_start': None, 
                'checkpoint': checkpoint}

with open(filename,'rb') as fp:
        data = pickle.load(fp)
    
feature_map = data['feature_map']
cardinalities_map = data['categorical_cardinalities']

/glade/u/home/ayal/.local/lib/python3.10/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator QuantileTransformer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/glade/u/home/ayal/.local/lib/python3.10/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
structure = {
            'num_historical_numeric': len(feature_map['historical_ts_numeric']),
            'num_historical_categorical': len(feature_map['historical_ts_categorical']),
            'num_static_numeric': len(feature_map['static_feats_numeric']),
            'num_static_categorical': len(feature_map['static_feats_categorical']),
            'num_future_numeric': len(feature_map['future_ts_numeric']),
            'num_future_categorical': len(feature_map['future_ts_categorical']),
            'historical_categorical_cardinalities': [cardinalities_map[feat] + 1 for feat in feature_map['historical_ts_categorical']],
            'static_categorical_cardinalities': [cardinalities_map[feat] + 1 for feat in feature_map['static_feats_categorical']],
            'future_categorical_cardinalities': [cardinalities_map[feat] + 1 for feat in feature_map['future_ts_categorical']],
        }

configuration['data_props'] = structure




is_cuda = torch.cuda.is_available()
device = torch.device("cuda" if is_cuda else "cpu")

In [4]:
model = TemporalFusionTransformer(config=OmegaConf.create(configuration))

In [5]:
state_dict = torch.load(checkpoint, map_location=device)
model_state = model.state_dict()
model.load_state_dict(state_dict, strict=False)

<All keys matched successfully>

In [6]:
# 3) save & reload
tmp_path = "/glade/u/home/ayal/phenology-ml-clm/models/"
out_file = tmp_path+"tft_scripted.pt"
model.eval()
model_scripted = torch.jit.script(model)

RuntimeError: 
Return value was annotated as having type Tensor but is actually of type List[Tensor]:
  File "/glade/u/home/ayal/tft-torch/tft_torch/base_blocks.py", line 18
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.dim() <= 2:
            return self.module(x)
            ~~~~~~~~~~~~~~~~~~~~ <--- HERE
    
        # Squash samples and timesteps
